In [4]:
%load_ext autoreload
%autoreload 2

from src.cleaner.auto_cleaner import clean_text,ml_clean_text
import glob
import json
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#クリーニングする処理
text="衝撃!!|中山トンネルは、上越新幹線高崎駅 - 上毛高原駅間にある総延長14,857 mの複線鉄道トンネルであり高崎方面から進行すると榛名トンネルの次、二番目に通過するトンネルである。建設中に2回の大出水事故を起こして難工事を極め、2回の経路変更によりようやく完成したが、経路変更のためにトンネル内に半径1,500 mの曲線ができてしまい、営業速度240 km/hの新幹線がトンネル内の曲線部分を通過するときには160 km/hに減速せざるをえなくなった。"
#text="中山トンネルは、上越新幹線高崎駅 - 上毛高原駅間にある総延長14,857 mの複線鉄道トンネルであり高崎方面から進行すると榛名トンネルの次、二番目に通過するトンネルである。建設中に2回の大出水事故を起こして難工事を極め、2回の経路変更によりようやく完成したが、経路変更のためにトンネル内に半径1,500 mの曲線ができてしまい、営業速度240 km/hの新幹線がトンネル内の曲線部分を通過するときには160 km/hに減速せざるをえなくなった。"
print(clean_text(text))

中山トンネルは、上越新幹線高崎駅-上毛高原駅間にある総延長14,857 mの複線鉄道トンネルであり高崎方面から進行すると榛名トンネルの次、二番目に通過するトンネルである。建設中に2回の大出水事故を起こして難工事を極め、2回の経路変更によりようやく完成したが、経路変更のためにトンネル内に半径1,500 mの曲線ができてしまい、営業速度240 km/hの新幹線がトンネル内の曲線部分を通過するときには160 km/hに減速せざるをえなくなった。


In [6]:
#以下､クラスタリング後のjsonlファイルを読み込む例｡
#datasetsライブラリから､適当にmc4-jaなどを読み込んで問題ナシ｡
path_list=glob.glob("../data/categorized/**/*.jsonl",recursive=True)


In [7]:
len(path_list)

2124694

In [8]:
random.shuffle(path_list)

In [9]:
line_list=[]
path=random.choice(path_list)
with open(path,"r") as f:
    #1行ずつ読み込む
    for line in f:
        text=json.loads(line)["text"]
        cleaned=clean_text(text)
        print(cleaned)
        break

Instagramのビジネス利用におけるマル秘テクニックを伺いました!


In [10]:
text

'Instagramのビジネス利用におけるマル秘テクニックを伺いました!'

In [11]:
print(clean_text(text))

Instagramのビジネス利用におけるマル秘テクニックを伺いました!


In [12]:
print(text)

Instagramのビジネス利用におけるマル秘テクニックを伺いました!


In [13]:

# %%
with open ("temp/gz_list.txt","r") as f:
    gz_list = f.read().splitlines()

gz_list=[i for i in gz_list if i.endswith('.gz')]
print(len(gz_list), " files found")



3732  files found


In [14]:

from src.load_gz import read_gzip_json_file
database_path=gz_list[0]

lines=[]
for article in read_gzip_json_file(database_path):
    text = article.get('text', '')  # 'text'キーからテキストデータを取得
    lines.append(text)



In [15]:
lines[1]

database_name = database_path.split("/")[-1]  # .split(".")[0]

database_name = database_name.replace(".json", "").replace(".gz", "")
database_name

'jap2010_0069_0l'

In [16]:
import argparse
import json
import os
from src.classify.Text2Vec import Text2Vec, texts2classes
from src.cleaner.auto_cleaner import clean_text, ml_clean_text
from gensim.models.fasttext import load_facebook_model
import joblib
from src.load_gz import read_gzip_json_file

streaming = True
base_dir = "../data/categorized"
length_threshold = 30  # 短い記事は捨てる
check_length = 200  # はじめのlengthだけで分類する

# 機械学習で記事を選別する
do_ml_clean = True

# load models
t2v = Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))
kmeans = joblib.load("../data/model/kmeans.pkl")


def make_dir(path):
    if not os.path.exists(path):
        os.mkdir(path)


make_dir(base_dir)

In [21]:


# %%

# %%
batch_size = 100


def proc(docs):
    # docsを処理する関数
    # ここに処理のロジックを実装します
    print(f"Processing {len(docs)} documents...")
    categories = texts2classes(docs, t2v, kmeans, length=check_length)

    for text, category in zip(docs, categories):
        save_dir = f"{base_dir}/{category}"
        make_dir(save_dir)
        database_name = database_path.split("/")[-1]  # .split(".")[0]

        data = json.dumps(
            {
                # "db": database_name, #特に必要ない｡storage 節約
                "text": text},
            ensure_ascii=False
        )
        database_name = database_name.replace(".jsonl", "").replace(".gz", "")
        print(f"Saving {save_dir}/{database_name}.jsonl")
        with open(f"{save_dir}/{database_name}.jsonl", "a") as f:
            f.write(data+"\n")

    return len(docs)


def main():

    docs = []

    lines = []
    for article in read_gzip_json_file(database_path):
        text = article.get('text', '')  # 'text'キーからテキストデータを取得
        lines.append(text)

    for text in lines:
        if do_ml_clean:
            text = ml_clean_text(text)
        else:
            text = clean_text(text)
        if len(text) < length_threshold:
            continue

        docs.append(text)
        if len(docs) == batch_size:
            # docsのコピーを作成してprocに渡す
            proc(docs[:])
            # docsをリセット
            docs = []


#if __name__ == "__main__":
#    main()

In [22]:
docs=[]
batch_size=100
for text in lines:
    if do_ml_clean:
        text = ml_clean_text(text)
    else:
        text = clean_text(text)
    if len(text) < length_threshold:
        continue

    docs.append(text)
    if len(docs) == batch_size:
        # docsのコピーを作成してprocに渡す
        proc(docs[:])
        break
        # docsをリセット
        docs = []

Processing 100 documents...
Saving ../data/categorized/2497/jap2010_0069_0.jsonl
Saving ../data/categorized/3102/jap2010_0069_0.jsonl
Saving ../data/categorized/7228/jap2010_0069_0.jsonl
Saving ../data/categorized/7026/jap2010_0069_0.jsonl
Saving ../data/categorized/3596/jap2010_0069_0.jsonl
Saving ../data/categorized/5920/jap2010_0069_0.jsonl
Saving ../data/categorized/5593/jap2010_0069_0.jsonl
Saving ../data/categorized/3803/jap2010_0069_0.jsonl
Saving ../data/categorized/1353/jap2010_0069_0.jsonl
Saving ../data/categorized/8375/jap2010_0069_0.jsonl
Saving ../data/categorized/2560/jap2010_0069_0.jsonl
Saving ../data/categorized/2645/jap2010_0069_0.jsonl
Saving ../data/categorized/6534/jap2010_0069_0.jsonl
Saving ../data/categorized/9661/jap2010_0069_0.jsonl
Saving ../data/categorized/1017/jap2010_0069_0.jsonl
Saving ../data/categorized/6059/jap2010_0069_0.jsonl
Saving ../data/categorized/4893/jap2010_0069_0.jsonl
Saving ../data/categorized/2980/jap2010_0069_0.jsonl
Saving ../data/cat